In [45]:
# model and data are ready
# it's time to train, validate and test the model
# training is an iterative process, in each iteration
# > the model guesses an output 
# > calculates the error with the guess through loss function
# > collects the derivatives of error (retrieve the gradients of parameters)
# > optimizes parameters using the gradient descent

In [46]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [47]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [48]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [49]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)

In [50]:
class LeNet(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1)
        # Fully connected layers
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        # First convolutional layer + activation + pooling
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        # Second convolutional layer + activation + pooling
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        # Flatten
        x = x.view(x.size(0), -1)
        # Fully connected layers + activation
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # Output layer
        x = self.fc3(x)
        return x
    
model = LeNet().to(device)

In [51]:
learning_rate = 1e-3

In [52]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # set mode as training: important for batch normalization and dropout layers
    model.train() # unnecessary in this situation, but added for best practice
    for batch, (X, y) in enumerate(dataloader):
        # move data to device
        X, y = X.to(device), y.to(device)
        # forward
        pred = model(X)
        loss = loss_fn(pred, y)
        # back propagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # # print loss every 100 batches
        # if batch%100 == 0:
        #     loss, current = loss.item(), batch*len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    # set mode as evaluation: important for batch normalization and dropout layers
    model.eval() # unnecessary in this situation, but added for best practice
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # disable gradient computing
    with torch.no_grad():
        for X, y in dataloader:
            # move data to device
            X, y = X.to(device), y.to(device)
            # forward
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            # calculate accuracy
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test --- Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")


In [54]:
# the main
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10 # read as epics, means an entire training dataset through the learning algorithm
for t in range(epochs):
    # print(f"Epoch {t+1}\n" + '-'*20)
    print(f"Epoch {t+1} ", end='')
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

# On Macbook M3: did 10+100+1000+10000 epochs, got Accuracy: 88.8%, Avg loss: 0.925
# On Desktop 4090: trained for 15 min, got Accuracy: 89.0%, Avg loss: 0.314
# On Desktop 4090: trained for 1 hour, got Accuracy: 89.2%, Avg loss: 0.314

Epoch 1 Test --- Accuracy: 10.0%, Avg loss: 2.298566

Done!


In [ ]:
# save the model
torch.save(model.state_dict(), "5_weights.pth")